# Importing libraries

In [ ]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Displaying an image from dataset

In [ ]:
imgSrc = "Prediction_Testing/open eyes/s0007_00614_0_0_1_0_0_01.png"
img = cv2.imread(imgSrc)

img_cvt = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img_cvt)
plt.show()

In [ ]:
img_cvt.shape

# Setting up dataset

In [ ]:
print(os.path.join("Prediction_Testing/", "close eyes"))
print(len(os.listdir(os.path.join("Prediction_Testing/", "close eyes"))))
print(len(os.listdir(os.path.join("Prediction_Testing/", "open eyes"))))

In [ ]:
# TO EXTRACT AND TRAIN DATA LOCALLY
Datadirectory = "Prediction_Testing/"
Classes = ["close eyes", "open eyes"]

In [ ]:
for category in Classes:
  path = os.path.join(Datadirectory, category)
  for img in os.listdir(path):
    img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
    backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
    plt.imshow(img_array, cmap = "gray")

In [ ]:
img_size = 224

new_array = cv2.resize(backtorgb,(img_size, img_size))
plt.imshow(new_array, cmap="gray")
plt.show()

# Converting images into an array for data and labels

In [ ]:
# Reading all images and converting them into an array for data and labels

img_size = 224
training_Data = []

def creating_training_Data():
  for category in Classes:
    path = os.path.join(Datadirectory, category)
    class_num = Classes.index(category) # 0 1 
    for img in os.listdir(path):
      try:
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
        backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
        new_array = cv2.resize(backtorgb, (img_size, img_size))
        training_Data.append([new_array, class_num])
      except Exception as e:
        print(e)
        pass

creating_training_Data()

In [ ]:
# print(training_Data)
print(len(training_Data))

# Storing X and Y labels using 'Pickle'

In [ ]:
import random

In [ ]:
# Shuffles all the dataset images
random.shuffle(training_Data)

In [ ]:
# making dataset eligible for transfer learning
X = []
y = []

for features,label in training_Data:
  X.append(features)
  y.append(label)

X = np.array(X).reshape(-1, img_size, img_size, 3)

In [ ]:
X.shape

In [ ]:
# normalize the date
X = X / 255.0; 
y = np.array(y)

In [ ]:
import pickle

In [ ]:
pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out, protocol=4)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out, protocol=4)
pickle_out.close()

In [ ]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

# all labels in X and all frames are in y now

In [ ]:
print(X)

In [ ]:
print(y)

# Training & saving model

In [ ]:
model = tf.keras.applications.mobilenet.MobileNet()

In [ ]:
model.summary()

In [ ]:
base_input = model.layers[0].input # input
base_output = model.layers[-4].output # dropout

In [ ]:
Flat_layer = layers.Flatten()(base_output)
final_output = layers.Dense(1)(Flat_layer)

final_output = layers.Activation('tanh')(final_output)

In [ ]:
new_model = keras.Model(inputs=base_input, outputs=final_output)

In [ ]:
new_model.summary()

In [ ]:
new_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) # Adaptive Moment Estimation (ADAM)

In [ ]:
# model fitting/training
new_model.fit(X, y, epochs=6, validation_split=0.30)

In [ ]:
new_model.save('drowsiness_model.h5')

# Loading trained model & prediction on test image

In [ ]:
new_model = tf.keras.models.load_model('drowsiness_model.h5')

In [ ]:
img_size = 224

imgSrc = "Prediction_Testing/s0001_02334_0_0_1_0_0_01.png"
# imgSrc = "Prediction_Testing/s0001_00001_0_0_0_0_0_01.png"

img_array = cv2.imread(imgSrc, cv2.IMREAD_GRAYSCALE)
backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
new_array = cv2.resize(backtorgb, (img_size, img_size))

In [ ]:
X_input = np.array(new_array).reshape(1, img_size, img_size, 3)

In [ ]:
X_input.shape

In [ ]:
plt.imshow(new_array)

In [ ]:
X_input = X_input / 255.0

In [ ]:
prediction = new_model.predict(X_input)

# -ve -> closed eye
# +ve -> opened eye
print(prediction)

result = "Opened eyes detected" if prediction > 0 else "Closed eyes detected"
print(result)